In [1]:
import sys

sys.path.append("../../")

from models.Majority_vote import MV
from stats import evaluate_perf_utils as eval_perf
import utils.load_csv as load_csv

2024-12-08 23:59:46.825951: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733698786.843921  200252 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733698786.849157  200252 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-08 23:59:46.869349: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/thibaud/majority-vote-outlier-detection/.venv/lib/python3.12/site-packages/torchvision/transforms/v2/_deprecated.py:42:

In [10]:
import os
print(os.getcwd())

mv = MV.Majority_Vote(som_model_path="../../models/SOM/hpc_SOM_model.pkl", vae_model_path="../../models/VAE/vae.pth", cnn_model_path="../../models/CNN/cnn.keras") 
all_data = load_csv.load_pandas()
_, val_set, test_set = load_csv.split_data(all_data)
mv.set_thresholds(val_set)
predictions = mv.predict(test_set)

eval_perf.evaluate(mv.cnn.y_test, test_set)



/home/thibaud/majority-vote-outlier-detection/models/Majority_vote
Model loaded from ../../models/SOM/hpc_SOM_model.pkl.


/home/thibaud/majority-vote-outlier-detection/models/Majority_vote/../../utils/load_save_models.py:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(to

Threshold set at 2.5560 (Percentile: 90)


AttributeError: 'NoneType' object has no attribute 'load_data'